In [1]:
from datasets import load_from_disk

re_dataset = load_from_disk('../datasets/intermediate/re_dataset_url/').shuffle(seed=42)

<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow._fs.FileInfo size changed, may indicate binary incompatibility. Expected 64 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow._fs.FileSelector size changed, may indicate binary incompatibility. Expected 48 from C header, got 72 from PyObject


In [2]:
re_dataset

DatasetDict({
    test: Dataset({
        features: ['url', 'entities', 'paragraph'],
        num_rows: 7721
    })
    validation: Dataset({
        features: ['url', 'entities', 'paragraph'],
        num_rows: 15126
    })
    train: Dataset({
        features: ['url', 'entities', 'paragraph'],
        num_rows: 120323
    })
})

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
from transformers import BitsAndBytesConfig
import torch

model_id = "mistralai/Mistral-7B-v0.1"
peft_model_id = '../finetune/qlora-out-relation-extractionv5'

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_4bit=True,
    device_map='auto',
    torch_dtype=torch.bfloat16,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4'
    ),
)
model = PeftModel.from_pretrained(model, peft_model_id)

tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

CUDA extension not installed.
CUDA extension not installed.


In [4]:
def parse_relations(output):
    if output == 'Nincs kapcsolat.':
        return []
    relations = []
    for section in output.split('\n\n'):
        lines = section.split('\n')
        if len(lines) != 4:
            return None
        if not (lines[0].startswith('Indoklás: ') or
                lines[1].startswith('Kapcsolat: ') or
                lines[2].startswith('Tárgy: ') or
                lines[3].startswith('Alany: ')):
            return None
        relations.append({
            "explanation": lines[0].replace('Indoklás: ', '').replace('"', ''),
            "relation": lines[1].replace('Kapcsolat: ', '').replace('"', ''),
            "subject": lines[2].replace('Tárgy: ', '').replace('"', ''),
            "object": lines[3].replace('Alany: ', '').replace('"', ''),
        })
    return relations

In [5]:
def format_input(paragraph):
    text = ''
    text += 'Bekezdés:\n"' + paragraph.replace('"', "'") + '"\n'
    return text.strip()+'\n\nRelációk:\n'


In [7]:
from tqdm import tqdm

device = 'cuda:0'
preds = []

for row in tqdm(re_dataset['test']):
    text = format_input(row['paragraph'])
    url = format_input(row['url'])
    with torch.cuda.amp.autocast():
        inputs = tokenizer(text, return_tensors="pt").to(device)
        if len(inputs['input_ids'][0]) > 2048:
            continue
        outputs = model.generate(input_ids=inputs['input_ids'], max_new_tokens=1024)
        result = tokenizer.decode(outputs[0], skip_special_tokens=True)
        oresult = result.split('Relációk:\n')[1].strip()
        print(result)
        preds.append({'relations': parse_relations(oresult), 'paragraph': row['paragraph'], 'url': url})


  0%|                                                                                                                                                    | 0/7721 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  0%|                                                                                                                                         | 1/7721 [00:16<35:50:52, 16.72s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Bekezdés:
"A kormány erős embere, Lázár János Miniszterelnökséget vezető miniszter még több mindent tudott kilobbizni Hódmezővásárhelynek: a stadion és a bicikliút kötelező körnek számít, ahogy az uszoda is, de a jövő évi utánpótlás úszó-Eb-vel, a leendő vívó- és bokszcentrummal olyan extrákat ad a hódmezővásárhelyieknek, amelyeket más politikusoknak nem sikerült elérni. Lázár óvatos becslések szerint is 12 milliárd állami és uniós pénzt vonzott a városába, majdnem annyit, mint a közel kétszer akkora Szombathely, Hende Csaba honvédelmi miniszter büszkesége. Hende túl sok mindent nem tudott kiharcolni: 9,6 milliárdért új stadiont kap a város, de ez inkább a népszerű focicsapatnak, a Haladásnak köszönhető, mintsem a honvédelmi miniszternek."

Relációk:
 Indoklás: "A bekezdés szerint Lázár János Miniszterelnökséget vezető miniszter, tehát közöttük egy vezetői kapcsolat áll fenn."
Kapcsolat: "vezető"
Tárgy: "Lázár János"
Alany: "Miniszterelnökség"

Indoklás: "A bekezdés szerint Hende Csaba

  0%|                                                                                                                                         | 2/7721 [00:27<28:01:11, 13.07s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Bekezdés:
"Ő állítása szerint ezt ott helyben elutasította. De a koordinátor szerint a Novelmedix kereste meg őt lényegében azzal, hogy a Novelmedix is beszállhasson a kartellbe, és szigorúan nem is volt ott a Siemenstől senki."

Relációk:
 Indoklás: "A bekezdés szerint a Novelmedix kereste meg a koordinátort, hogy beszállhasson a kartellbe. Tehát a Novelmedix és a koordinátor között egy kommunikációs kapcsolat áll fenn."
Kapcsolat: "kommunikáció"
Tárgy: "Novelmedix"
Alany: "Koordinátor"


  0%|                                                                                                                                         | 3/7721 [00:38<26:44:38, 12.47s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Bekezdés:
"Befektetési csalás miatt eljárás Rónai ellen Rónai Attiláról a Népszabadság 2010. április 2-án megírta, hogy már a kilencvenes években felfigyelt a rendőrség, mint az olajügyek kulcsfigurái körül feltűnő alakra: testőrként, sofőrként tevékenykedett és olykor pénzszállításokat végeztettek vele. A Népszabadság értesülése szerint 2006-2007-ben elkövetett, 16 millió forint kárt okozó befektetési csalás gyanúsítottjaként nyomozott ellene a rendőrség, és ügye 2010 tavaszán ügyészi szakaszba ért."

Relációk:
 Indoklás: "A bekezdés szerint Rónai Attila testőrként és sofőrként tevékenykedett, tehát közöttük munkakapcsolat áll fenn."
Kapcsolat: "munkakapcsolat"
Tárgy: "Rónai Attila"
Alany: "testőr, sofőr"

Indoklás: "A bekezdés szerint Rónai Attila gyanúsított volt befektetési csalás miatt, tehát közöttük ügyészségi kapcsolat áll fenn."
Kapcsolat: "ügyészségi kapcsolat"
Tárgy: "Rónai Attila"
Alany: "befektetési csalás gyanúsítottja"


  1%|▉                                                                                                                                       | 50/7721 [15:51<44:55:13, 21.08s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Bekezdés:
"Hernádi és a csúszópénzek ügye ennél kevésbé világos. Hogy mi történt valójában, arról jövő hét kedden hirdet ítéletet első fokon a bíróság. A vád szerint Hernádi Zsolt tízmillió eurót fizetett Sanadernek, hogy a Mol megkapja a horvát olajvállalat irányítási jogát, illetve úgy vehesse át a cég 47,26 százalékát, hogy a horvát állam leválasztja a cégről a veszteséges gázágazatot."

Relációk:
 Indoklás: "A bekezdés szerint Hernádi Zsolt fizetett Sanadernek, hogy a Mol megkapja a horvát olajvállalat irányítási jogát. Tehát Hernádi Zsolt és a Mol között üzleti kapcsolat áll fenn."
Kapcsolat: "üzleti"
Tárgy: "Hernádi Zsolt"
Alany: "Mol"

Indoklás: "A bekezdés szerint Hernádi Zsolt fizetett Sanadernek, hogy a horvát állam leválasztja a cégről a veszteséges gázágazatot. Tehát Hernádi Zsolt és a horvát állam között üzleti kapcsolat áll fenn."
Kapcsolat: "üzleti"
Tárgy: "Hernádi Zsolt"
Alany: "horvát állam"


  1%|▉                                                                                                                                       | 50/7721 [16:10<41:22:28, 19.42s/it]


KeyboardInterrupt: 

In [8]:
n = 0

In [9]:
print(n)
print(preds[n]['paragraph'])
print(preds[n]['relations'])
n += 1

0
A kormány erős embere, Lázár János Miniszterelnökséget vezető miniszter még több mindent tudott kilobbizni Hódmezővásárhelynek: a stadion és a bicikliút kötelező körnek számít, ahogy az uszoda is, de a jövő évi utánpótlás úszó-Eb-vel, a leendő vívó- és bokszcentrummal olyan extrákat ad a hódmezővásárhelyieknek, amelyeket más politikusoknak nem sikerült elérni. Lázár óvatos becslések szerint is 12 milliárd állami és uniós pénzt vonzott a városába, majdnem annyit, mint a közel kétszer akkora Szombathely, Hende Csaba honvédelmi miniszter büszkesége. Hende túl sok mindent nem tudott kiharcolni: 9,6 milliárdért új stadiont kap a város, de ez inkább a népszerű focicsapatnak, a Haladásnak köszönhető, mintsem a honvédelmi miniszternek.
[{'explanation': 'A bekezdés szerint Lázár János Miniszterelnökséget vezető miniszter, tehát közöttük egy vezetői kapcsolat áll fenn.', 'relation': 'vezető', 'subject': 'Lázár János', 'object': 'Miniszterelnökség'}, {'explanation': 'A bekezdés szerint Hende Cs